In [105]:
import pandas as pd 
import sqlite3
from datetime import datetime, timedelta
import matplotlib.pyplot as plt 

In [106]:
db_path = "/Users/yuta/Desktop/climb_proj/instance/climbs.db"

connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('users',), ('climbs',), ('score',), ('comments',)]


In [107]:
import os 
print(os.path.abspath(db_path))

/Users/yuta/Desktop/climb_proj/instance/climbs.db


In [108]:
class DataFrame:
    def __init__(self):
        query_climb = 'SELECT * FROM climbs'
        self.climbs_df = pd.read_sql_query(query_climb, connection)

        query_user = 'SELECT * FROM users'
        self.user_df = pd.read_sql_query(query_user, connection)

        query_score = 'SELECT * FROM score'
        self.score_df = pd.read_sql_query(query_score, connection)

df = DataFrame()
df.user_df


,id,user_name,user_icon,email,password,current_grade,home_gym
0,1,JohnDoe,/static/icons/johndoe_icon.png,john.doe@example.com,password123,V5,Mountain Gym
1,2,JaneSmith,/static/icons/janesmith_icon.png,jane.smith@example.com,password123,V3,Rock Center
2,3,aa,None,aa,pbkdf2:sha256:600000$DSPc8bd3$e39f15b0afd3fc6f...,5,
3,4,ww,None,ww,pbkdf2:sha256:600000$hQF9UJkz$c768d9e258d24278...,5,
4,5,ss,None,ss,pbkdf2:sha256:600000$Hgd4eTdB$bc7f775209cf8975...,1,
5,6,zz,None,zzz,pbkdf2:sha256:600000$YKDKCg6u$4ec2bc695167b780...,5,


In [109]:
# this returns today's climbing activity of user with specified id
def excercise_today(id):
    df = DataFrame()
    user_climb_data = df.climbs_df[df.climbs_df.upload_author_id==id] # make the value inside upload_author_id dynamic
    user_climb_data['date_logged'] = pd.to_datetime(user_climb_data.date_logged) # convert the value to datetime format for conpatibility
    user_data_today = user_climb_data[user_climb_data['date_logged'].dt.date == datetime.now().date()]
    if user_data_today.empty: # need to implement error handling in case there is no excersize history that day
        return None
    else:
        return user_data_today
    
# climbs_completed = excercise_today(3)[excercise_today(3).completed == True]
# climbs_completed

In [110]:
# this returns max_grade of the user with specified id 
def user_max(id):
    df = DataFrame()
    user_detail = df.user_df[df.user_df.id == id]
    return int(user_detail.current_grade)

user_max(3)

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3681893202.py:5: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(user_detail.current_grade)


5

In [111]:
# this returns performance score based solely on the attempt number and the grade which is before being refined to the final score 
def performance_score(id):
    max_attempts = 100
    grade = excercise_today(id)[excercise_today(id).completed == True].grade # this ensures that its getting only completed climbs
    user_max_grade = user_max(id)
    user_attempt = excercise_today(id)[excercise_today(id).completed == True].attempt # this ensures that its getting only completed climbs
    intensity_level = grade / user_max_grade
    score = intensity_level * (max_attempts / (user_attempt + max_attempts)) * 100
    return int(score.mean())

performance_score(3)


/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3976905911.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_climb_data['date_logged'] = pd.to_datetime(user_climb_data.date_logged) # convert the value to datetime format for conpatibility
/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3976905911.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_climb_data['date_logged'] = pd.to_datetime(user_climb_data.date_logged) # convert the value to datetime format for co

AttributeError: 'NoneType' object has no attribute 'completed'

In [112]:
def final_score(id):
    score = performance_score(id)
    user_max_grade = user_max(id)
    climbs_completed = excercise_today(id)[excercise_today(id).completed == True]
    completed_climb_num = climbs_completed.count().grade
    completed_climb_num

    # add or deduct number from perfromnace score based on the number of climbs completed
    if completed_climb_num == 1:
        score -= 10
    elif completed_climb_num == 2:
        score -= 8
    elif completed_climb_num == 3:
        score -= 6
    elif completed_climb_num == 4:
        score -= 4
    elif completed_climb_num == 5:
        score += 3
    elif completed_climb_num == 6:
        score += 5
    elif completed_climb_num > 6:
        score += 7

    # give extra score if the user cpmpleted climbs with the grade higher than the grade 2 grades lower than their max grade
    base_hard_climbs = climbs_completed[climbs_completed.grade >= user_max_grade -2].count().id
    score += base_hard_climbs * 2
    score

    # give extra score if the user cpmpleted their max grade
    max_grade = climbs_completed[climbs_completed.grade == user_max_grade].count().id
    score += max_grade * 3

    # give extra score if the user cpmpleted climbs with the grade higher than their max grade
    hard_grade = climbs_completed[climbs_completed.grade > user_max_grade].count().id
    score += hard_grade * 5
    
    if score > 100:
        score = 100
    return score

print(final_score(4))

9


/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3976905911.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_climb_data['date_logged'] = pd.to_datetime(user_climb_data.date_logged) # convert the value to datetime format for conpatibility
/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3976905911.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_climb_data['date_logged'] = pd.to_datetime(user_climb_data.date_logged) # convert the value to datetime format for co

In [113]:
# this returns table formatted data frame for graph for keeping track of score and corolies bunred throughout a week 
def score_and_caroly(id):
    df = DataFrame()
    score_df = df.score_df[df.score_df.score_owner_id == id]
    score_df['recorded_at'] = pd.to_datetime(score_df.recorded_at)
    score_df['date'] = score_df['recorded_at'].dt.date
    latest_per_day = score_df.loc[score_df.groupby('date')['recorded_at'].idxmax()]
    return latest_per_day

    # ax1 = plt.gca()  # Get the current axis
    # ax2 = ax1.twinx()  # Create a second y-axis sharing the same x-axis

    # # Set axis labels
    # ax1.set_ylabel('Performance Score', color='#E6232E')
    # ax2.set_ylabel('Calories Burned', color='skyblue')

    # # Plot data on respective axes
    # ax1.plot(latest_per_day.date, latest_per_day.performance_score, color='#E6232E')
    # ax2.plot(latest_per_day.date, latest_per_day.activity_score, color='skyblue')

    # # Set the y-axis minimum to 0 for both axes
    # ax1.set_ylim(bottom=0)  # Set min value for the primary axis (Performance Score)
    # ax2.set_ylim(bottom=0)  # Set min value for the secondary axis (Calories Burned)

    # # Display the plot
    # plt.show()
score_and_caroly(3)

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3245843569.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['recorded_at'] = pd.to_datetime(score_df.recorded_at)
/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3245843569.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['date'] = score_df['recorded_at'].dt.date


,id,performance_score,activity_score,grade,score_owner_id,recorded_at,date
6,7,78,479,8.0,3,2024-11-30 17:55:17.304365,2024-11-30
5,6,94,374,8.0,3,2024-12-01 17:55:17.304365,2024-12-01
4,5,100,163,9.0,3,2024-12-02 17:55:17.304365,2024-12-02
3,4,75,279,4.0,3,2024-12-03 17:55:17.304365,2024-12-03
2,3,97,202,4.0,3,2024-12-04 17:55:17.304365,2024-12-04
1,2,91,217,7.0,3,2024-12-05 17:55:17.304365,2024-12-05
0,1,73,286,8.0,3,2024-12-06 17:55:17.304365,2024-12-06
22,23,87,1290,1.0,3,2024-12-12 21:09:03.540487,2024-12-12
47,48,100,1960,3.3,3,2024-12-13 15:09:10.296907,2024-12-13
48,49,9,20,1.0,3,2024-12-21 13:45:46.888612,2024-12-21


In [ ]:
# returns average grade completed in the past 7 days 
def weekly_average_grade(id):  
    df = DataFrame()
    score_df = df.score_df[df.score_df['score_owner_id'] == id]
    score_df['recorded_at'] = pd.to_datetime(score_df['recorded_at'])
    score_df['date'] = score_df['recorded_at'].dt.date
    latest_per_day = score_df.loc[score_df.groupby('date')['recorded_at'].idxmax()]

    # Filter for the last 7 days
    one_week_ago = datetime.now().date() - timedelta(days=7)
    last_week_data = latest_per_day[latest_per_day['date'] >= one_week_ago]
    mean_grade = last_week_data['grade'].mean()
    return mean_grade


In [ ]:
# this retunrs table format data frame for all climbs complete by grade 
def all_completed_grade(id):
    df = DataFrame()
    user_climbs = df.climbs_df[df.climbs_df['upload_author_id'] == id]
    filtered_users_climbs = user_climbs[user_climbs['completed'] == True]
    total_grade_completed = filtered_users_climbs.groupby('grade').count()
    return total_grade_completed

    # plt.bar(
    #     total_grade_completed.index,  # Labels for each style (on x-axis)
    #     total_grade_completed['id']   # Values for each style (on y-axis)
    # )

    # # Add a title
    # plt.title('All Climbs Completed by Grade')

    # # Display the chart
    # plt.show()

all_completed_grade(3)

,id,photo,completed,date_completed,style,evaluation,when_stripped,is_stripped,date_logged,attempt,total_attempts,upload_author_id,feel_strong
grade,,,,,,,,,,,,,
1,20,20,20,20,20,20,0,20,20,20,20,20,0
2,1,1,1,1,1,1,0,1,1,1,1,1,0
3,4,4,4,4,4,4,0,4,4,4,4,4,0
4,5,5,5,5,5,5,1,5,5,5,5,5,1
5,5,5,5,5,5,5,0,5,5,5,5,5,1
6,2,2,2,2,2,2,0,2,2,2,2,2,2
7,1,1,1,1,1,1,0,1,1,1,1,1,1
8,3,3,3,3,3,3,0,3,3,3,3,3,1
10,1,1,1,1,1,1,0,1,1,1,1,1,0


In [ ]:
# this returns table formated data frame for style of all the climbs completed
def style_completed(id):
    df = DataFrame()
    user_climbs = df.climbs_df[df.climbs_df['upload_author_id'] == id]
    # print(user_climbs)
    total_completed = user_climbs[user_climbs.completed == True]
    total_completed = total_completed.groupby('style').count()
    by_style = total_completed[['id']]
    return by_style

   # Create the bar chart
    # plt.figure(figsize=(10, 6))  # Set figure size
    # plt.barh(
    #     by_style.index,  # Climb styles on y-axis
    #     by_style['id'],  # Number of attempts on x-axis
    #     color='skyblue'  # Bar color
    # )
    # plt.xlabel('Completed Num', fontsize=12)
    # plt.ylabel('Genra', fontsize=12)
    # plt.title('All Climbing Styles Completed', fontsize=14)
    # plt.tight_layout()
    # plt.show()

style_completed(3)

,id
style,
Balance,1
Climp,3
Cordination,3
Endurance,1
Finger,24
FootTech,2
Jug,1
Pinch,2
Pocket,2


In [ ]:
# this returns a table format data with styles and number of climbs in each styles user attempted today 
def style_attempted(id):
    df = DataFrame()
    user_climbs = df.climbs_df[df.climbs_df['upload_author_id'] == id]
    user_climbs['date_logged'] = pd.to_datetime(user_climbs.date_logged).dt.date
    total_today = user_climbs[user_climbs.date_logged == datetime.now().date()]
    total_today = total_today.groupby('style').count()
    by_style = total_today[['id']]

    json_output = [{style: row['id']} for style, row in by_style.iterrows()]
    return json_output

style_attempted(3)

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/4018518453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_climbs['date_logged'] = pd.to_datetime(user_climbs.date_logged).dt.date


[{'Finger': np.int64(1)}]

In [ ]:
# this returns tabel formatted data frame for climbs attempted today by grade 
def grade_attempted(id):
    df = DataFrame()
    user_climbs = df.climbs_df[df.climbs_df['upload_author_id'] == id]
    user_climbs = user_climbs[user_climbs.completed == True]
    user_climbs['date_logged'] = pd.to_datetime(user_climbs.date_logged).dt.date
    total_today = user_climbs[user_climbs.date_logged == datetime.now().date()]
    filtered_total_today = total_today[['grade', 'attempt']]
    return filtered_total_today
    # print(filtered_total_today)
    # plt.bar(
    #     filtered_total_today['grade'],  # X-axis: grade values
    #     filtered_total_today['attempt']  # Y-axis: attempt values
    # )

    # # Add labels and title
    # plt.xlabel('Grade')  # Label for the x-axis
    # plt.ylabel('Number of Attempts')  # Label for the y-axis
    # plt.title('Grade Attempted today')  # Title of the chart
    # plt.xticks(ticks=filtered_total_today['grade'], labels=filtered_total_today['grade'])

    # # Display the chart
    # plt.show()
grade_attempted(3)

,grade,attempt
49,1,2


In [ ]:
def get_cal(id):
    user_score = df.score_df[df.score_df['score_owner_id'] == id]
    if not user_score.empty:
        latest_data = user_score.loc[0]
        latest_data['recorded_at'] = pd.to_datetime(latest_data.recorded_at).date()
        if not latest_data.recorded_at == datetime.now().date():
            return 0
        else:
            return latest_data.performance_score
    else:
        return 0
get_cal(3)

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/2538885710.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_data['recorded_at'] = pd.to_datetime(latest_data.recorded_at).date()


0

In [ ]:
def all_climbs_num(id):
    all_climbs_registered = df.climbs_df[df.climbs_df.upload_author_id == id]
    return all_climbs_registered.count().id
all_climbs_num(3)

np.int64(43)

In [ ]:
def climbs_completed(id):
    all_climbs_registered = df.climbs_df[df.climbs_df.upload_author_id == id]
    return all_climbs_registered[all_climbs_registered.completed == True].count().id
climbs_completed(3)

np.int64(43)

In [ ]:
score_and_caroly(3)

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3245843569.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['recorded_at'] = pd.to_datetime(score_df.recorded_at)
/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3245843569.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['date'] = score_df['recorded_at'].dt.date


,id,performance_score,activity_score,grade,score_owner_id,recorded_at,date
6,7,78,479,8.0,3,2024-11-30 17:55:17.304365,2024-11-30
5,6,94,374,8.0,3,2024-12-01 17:55:17.304365,2024-12-01
4,5,100,163,9.0,3,2024-12-02 17:55:17.304365,2024-12-02
3,4,75,279,4.0,3,2024-12-03 17:55:17.304365,2024-12-03
2,3,97,202,4.0,3,2024-12-04 17:55:17.304365,2024-12-04
1,2,91,217,7.0,3,2024-12-05 17:55:17.304365,2024-12-05
0,1,73,286,8.0,3,2024-12-06 17:55:17.304365,2024-12-06
22,23,87,1290,1.0,3,2024-12-12 21:09:03.540487,2024-12-12
47,48,100,1960,3.3,3,2024-12-13 15:09:10.296907,2024-12-13
48,49,9,20,1.0,3,2024-12-21 13:45:46.888612,2024-12-21


In [ ]:
score_df = df.score_df[df.score_df['score_owner_id'] == 3]
score_df['recorded_at'] = pd.to_datetime(score_df['recorded_at'])
score_df['date'] = score_df['recorded_at'].dt.date

# Get the latest record for each day
latest_per_day = score_df.loc[score_df.groupby('date')['recorded_at'].idxmax()]
latest_per_day

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/1612406857.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['recorded_at'] = pd.to_datetime(score_df['recorded_at'])
/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/1612406857.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['date'] = score_df['recorded_at'].dt.date


,id,performance_score,activity_score,grade,score_owner_id,recorded_at,date
6,7,78,479,8.0,3,2024-11-30 17:55:17.304365,2024-11-30
5,6,94,374,8.0,3,2024-12-01 17:55:17.304365,2024-12-01
4,5,100,163,9.0,3,2024-12-02 17:55:17.304365,2024-12-02
3,4,75,279,4.0,3,2024-12-03 17:55:17.304365,2024-12-03
2,3,97,202,4.0,3,2024-12-04 17:55:17.304365,2024-12-04
1,2,91,217,7.0,3,2024-12-05 17:55:17.304365,2024-12-05
0,1,73,286,8.0,3,2024-12-06 17:55:17.304365,2024-12-06
22,23,87,1290,1.0,3,2024-12-12 21:09:03.540487,2024-12-12
47,48,100,1960,3.3,3,2024-12-13 15:09:10.296907,2024-12-13
48,49,9,20,1.0,3,2024-12-21 13:45:46.888612,2024-12-21


In [114]:
score_df = df.climbs_df[df.climbs_df['upload_author_id'] == 3]
score_df['date_completed'] = pd.to_datetime(score_df['date_completed'])
score_df['date'] = score_df['date_completed'].dt.date

one_week_ago = datetime.now().date() - timedelta(days=7)
last_week_data = score_df[score_df['date'] >= one_week_ago]
mean_grade = round(last_week_data['grade'].mean(), 1)
mean_grade


/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/1356720061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['date_completed'] = pd.to_datetime(score_df['date_completed'])
/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/1356720061.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['date'] = score_df['date_completed'].dt.date


np.float64(1.0)

In [115]:
user_climbs = df.climbs_df[(df.climbs_df['upload_author_id'] == 4)&(df.climbs_df['attempt'] != 0)]
user_climbs['date_logged'] = pd.to_datetime(user_climbs['date_logged']).dt.date
today_climbs = user_climbs[user_climbs['date_logged'] == datetime.now().date()]
today_climbs = today_climbs.groupby('style').count()
by_style = today_climbs[['id']]
json_output = [{style: int(row['id'])} for style, row in by_style.iterrows()]
json_output
user_climbs

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/1163005566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_climbs['date_logged'] = pd.to_datetime(user_climbs['date_logged']).dt.date


,id,photo,completed,date_completed,grade,style,evaluation,when_stripped,is_stripped,date_logged,attempt,total_attempts,upload_author_id,feel_strong
56,70,/Users/yuta/Desktop/climb_proj/uploads/Screens...,1,2024-12-22,1,Finger,1.0,None,0,2024-12-22,1,1,4,NaN


In [ ]:
p_score = df.score_df[df.score_df['score_owner_id'] == 4].activity_score.iloc[-1]
p_score


np.int64(240)

In [ ]:
a = score_and_caroly(4)
a.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 54 to 54
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1 non-null      int64         
 1   performance_score  1 non-null      int64         
 2   activity_score     1 non-null      int64         
 3   grade              1 non-null      float64       
 4   score_owner_id     1 non-null      int64         
 5   recorded_at        1 non-null      datetime64[ns]
 6   date               1 non-null      object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 64.0+ bytes


/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3245843569.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['recorded_at'] = pd.to_datetime(score_df.recorded_at)
/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3245843569.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['date'] = score_df['recorded_at'].dt.date


In [ ]:
i = score_and_caroly(3)
i.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 6 to 48
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 10 non-null     int64         
 1   performance_score  10 non-null     int64         
 2   activity_score     10 non-null     int64         
 3   grade              10 non-null     float64       
 4   score_owner_id     10 non-null     int64         
 5   recorded_at        10 non-null     datetime64[ns]
 6   date               10 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 640.0+ bytes


/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3245843569.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['recorded_at'] = pd.to_datetime(score_df.recorded_at)
/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/3245843569.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['date'] = score_df['recorded_at'].dt.date


In [ ]:
user_climbs = df.climbs_df[(df.climbs_df['upload_author_id'] == 6)]
user_climbs['date_logged'] = pd.to_datetime(user_climbs['date_logged']).dt.date
today_completed = user_climbs[user_climbs['date_logged'] == datetime.now().date()]
filtered = today_completed[['id', 'completed', 'grade', 'attempt']]
filtered.to_dict(ori)

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/90244391.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[{'id': 14, 'completed': 1, 'grade': 7, 'attempt': 1},
 {'id': 24, 'completed': 1, 'grade': 7, 'attempt': 10},
 {'id': 72, 'completed': 0, 'grade': 6, 'attempt': 0}]

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/941877715.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/941877715.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_77386/941877715.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu